In [1]:
import os
import cv2
import random
import math
from collections import Counter
from tqdm.notebook import tqdm
import PIL.Image as Image
import numpy as np
import torch
import torchvision.transforms as transforms


from PIL import Image
import pandas as pd

import copy
import matplotlib.pyplot as plt


In [2]:
wsi_path = './images/'
mask_path = './masks/'
thumbnails_path = './thumbnails/'
transform_path=  './mask_transform/'
save_img_path = './init_data_image/'
save_init_path = './init_data/'
compress_path =  './compress/'

In [6]:
# 读取'image'文件夹里面的图片和'mask'文件夹里面的图片，将其对应的图片一起切片成224*224的图片，保存在'thumbnails'文件夹里面

import os
from PIL import Image

# 定义图片的尺寸和切片大小
image_size = (224, 224)
slice_size = 224

# 创建保存切片的文件夹
os.makedirs('./thumbnails/images', exist_ok=True)
os.makedirs('./thumbnails/masks', exist_ok=True)

# 读取'image'文件夹和'mask'文件夹中的图片
image_files = sorted(os.listdir('./images'))
mask_files = sorted(os.listdir('./masks'))

# 确保'image'文件夹和'mask'文件夹中图片数量相同
if len(image_files) != len(mask_files):
    print("Error: The number of images in 'image' folder is different from the number of images in 'mask' folder.")
    exit()

# 逐个处理每张图片
for i, (image_file, mask_file) in enumerate(zip(image_files, mask_files)):
    # 打开图片和对应的mask
    image_path = os.path.join('./images', image_file)
    mask_path = os.path.join('./masks', mask_file)
    image = Image.open(image_path)
    mask = Image.open(mask_path)

    # 确保图片和mask的尺寸相同
    if image.size != mask.size:
        print(f"Error: Image size doesn't match mask size for image {image_file}. Skipping this image.")
        continue

    # 计算切片的行数和列数
    rows = image.size[1] // slice_size
    cols = image.size[0] // slice_size

    # 逐个切片并保存
    for row in range(rows):
        for col in range(cols):
            # 计算切片的区域
            left = col * slice_size
            upper = row * slice_size
            right = left + slice_size
            lower = upper + slice_size

            # 裁剪图片和对应的mask
            image_slice = image.crop((left, upper, right, lower))
            mask_slice = mask.crop((left, upper, right, lower))

            # 保存切片图片和对应的mask
            slice_name = f'x_{row}_y_{col}_{i}.png'
            image_slice.save(os.path.join('thumbnails/images', slice_name))
            mask_slice.save(os.path.join('thumbnails/masks', slice_name))


In [5]:
# 创建保存切片的文件夹
os.makedirs('./init_data_image/images', exist_ok=True)
os.makedirs('./init_data_image/masks', exist_ok=True)
# 读取'image'文件夹和'mask'文件夹中的图片
images_files = sorted(os.listdir('./thumbnails/images'))
masks_files = sorted(os.listdir('./thumbnails/masks'))
# 将第前27张图片里面的切片提出来保存在 init_images文件夹里面
# 生成1到27的数组
num = np.arange(1,28)
for i in tqdm(range(len(images_files))):
    file_id = images_files[i].split('_')[4].split('.')[0]
    if int(file_id) in num:
        image_path = os.path.join('./thumbnails/images', images_files[i])
        mask_path = os.path.join('./thumbnails/masks', masks_files[i])
        image = Image.open(image_path)
        mask = Image.open(mask_path)
        image_file_change = file_id+'_x_'+images_files[i].split('_')[1]+"_y_"+images_files[i].split('_')[3]+".png"
        image.save(os.path.join('./init_data_image/images', image_file_change))
        mask.save(os.path.join('./init_data_image/masks', image_file_change))

100%|██████████| 48811/48811 [10:22<00:00, 78.40it/s]  


In [37]:
# 查看masks二值图像
# 读取./init_data_image/masks文件夹里面的第一张图片，打印出来，平切查看numpy格式
mask = Image.open('./masks/TCGA-A1-A0SK-DX1_xmin45749_ymin25055_MPP-0.2500.png')
# 将像素值为1的转成红色，其他的转成绿色，将这个改成RGB格式的图片
mask = mask.convert('RGB')
mask_arr = np.array(mask)
# 将像素值不为1的转成0
mask_arr[mask_arr!=1]=0
# 将mask_arr里面的像素值为1的转成红色，其他的转成绿色
colors=[
    [137,212,94], # 绿色
    [210,36,36], # 红色
    [240,65,69], # 
]
for i in range(mask_arr.shape[0]):
    for j in range(mask_arr.shape[1]):
        mask_arr[i,j,:]=colors[mask_arr[i,j,0]] # 将mask_arr里面的像素值为1的转成红色，其他的转成绿色
mask = Image.fromarray(mask_arr)


In [45]:
def transform_mask(mask,filename,mask_file):
    mask = mask.convert('RGB')
    mask_arr = np.array(mask)
    # 将像素值不为1的转成0
    mask_arr[mask_arr!=1]=0
    # 将mask_arr里面的像素值为1的转成红色，其他的转成绿色
    colors=[
        [137,212,94],
        [210,36,36],
        [240,65,69],
    ]
    for i in range(mask_arr.shape[0]):
        for j in range(mask_arr.shape[1]):
            mask_arr[i,j,:]=colors[mask_arr[i,j,0]]
    mask = Image.fromarray(mask_arr)
    path= f'./mask_transform/{filename}'
    os.makedirs(path, exist_ok=True)
    mask.save(path+f'/{mask_file}')

In [28]:
# 对图像进行癌症标签的分类

# 定义路径和标签字典
images_dir = './init_data_image/images'
masks_dir = './init_data_image/masks'
init_images_dir = './init_images'
labels_dict = {0: '0',1: '1', 2: '2', 3: '3'}

masks_files = sorted(os.listdir(masks_dir))
images_files = sorted(os.listdir(images_dir))

# 定义函数来计算面积比和获取标签
def get_label(mask):
    # 计算像素值为1的个数

    cancer_act_rate = [0.05, 0.15, 0.65, 0.9]
    interval=0.05

    num_1 = np.sum(mask == 1)
    area_ratio = num_1 / (224*224)
    if area_ratio > cancer_act_rate[0] and area_ratio < cancer_act_rate[1]:
        label = 1  # no cancer
    elif area_ratio > cancer_act_rate[1]+interval and area_ratio < cancer_act_rate[2]:
        label = 2  # cancer
    elif area_ratio > cancer_act_rate[2]+interval and area_ratio < cancer_act_rate[3]:
        label = 3  # more cancer
    else :
        label = 0 # 无效label
    return label




In [29]:
# 遍历所有切片图像
label1 = []
label2 = []
label3 = []
label0 = []
masks_dir = './init_data_image/masks'
images_dir = './init_data_image/images'
images_files=sorted(os.listdir(images_dir))
masks_files = sorted(os.listdir(masks_dir))
num_id = 1
middle_value = 0
for filename in tqdm(images_files):
    
    if filename.endswith('.png'):
        # 解析图像的行、列、切片编号
        filename_parts = filename.split('_')
        i, j, k = int(filename_parts[2]), int(filename_parts[4].split('.')[0]), int(filename_parts[0])
        
        if middle_value == k:
            pass
        else:
            os.makedirs(f'./init_images/image_{k}', exist_ok=True)
            filename_path = f'./init_images/image_{k}'
            middle_value = k
            num_id = 1
        # 读取切片图像和对应的masks图像
        image_path = os.path.join(images_dir, filename)
        image = np.array(Image.open(image_path))
        mask_path = os.path.join(masks_dir, filename)
        mask = np.array(Image.open(mask_path))
        # transform_mask(Image.open(mask_path),f'image{k}',filename)
        # 计算标签
        label = get_label(mask)
        if label == 1:
            label1.append(filename)
        elif label == 2:
            label2.append(filename)
        elif label == 3:
            label3.append(filename)
        else:
            label0.append(filename)
        # 保存图像和标签
        new_filename = f'{k}_{num_id}_{labels_dict[label]}.png'
        new_image_path = os.path.join(filename_path, new_filename)
        Image.fromarray(image).save(new_image_path)
        num_id+=1
# 打印label1、label2、label3的长度
print('无癌症数量：',len(label1),'\n 癌症数量：', len(label2),'\n 重度癌症数量：', len(label3))

100%|██████████| 7925/7925 [05:04<00:00, 26.01it/s]

无癌症数量： 348 
 癌症数量： 1551 
 重度癌症数量： 699


In [3]:

def npy_data_init_percent(
        img_path,  # /init_data_image
        save_path,  # /init_data
        label_rate=0.5,
        test_rate=0.2,
        size=224,
        img_num=0,
):

    if not os.path.exists(save_path):
        os.makedirs(save_path)
    # imglen是wsi图的个数
    train_dataset = []
    labeled_set = []
    unlabeled_set = []
    test_dataset = []
    files = []
    ts_files=[]


    transformations = transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
    )

    global_index = 0
    for imgid in tqdm(range(img_num)):
        files_path = []
        path = os.path.join(img_path, 'image_'+str(imgid+1))

        for file_name in os.listdir(path):
            lb=int(file_name.split('_')[-1][0]) 
            if lb!=0:
                files_path.append(file_name)

        patch_num = len(files_path)
        #
        rand_init = torch.randperm(patch_num)
        labeled_index = rand_init[:int(label_rate*patch_num)]
        unlabeled_index = rand_init[int(
            label_rate*patch_num): int((1-test_rate)*patch_num)]
        
        
        # labeled and unlabeled
        for patch_index in rand_init[:int((1-test_rate)*patch_num)]:
            files.append((imgid, patch_index, files_path[patch_index]))
            image = Image.open(os.path.join(path, files_path[patch_index]))
            new_data = Image.fromarray(np.uint8(image)).convert('RGB')
            img = transformations(new_data)
            label = int(files_path[patch_index][-5])
            train_dataset.append(
                (img, label, patch_index, imgid, global_index))
            if patch_index in labeled_index:
                labeled_set.append(
                    (img, label, patch_index, imgid, global_index))

            elif patch_index in unlabeled_index:
                unlabeled_set.append(
                    (img, label, patch_index, imgid, global_index))

            global_index += 1
        
        
        # test
        for patch_index in rand_init[int((1-test_rate)*patch_num):]:
            ts_files.append((imgid, patch_index, files_path[patch_index]))
            image = Image.open(os.path.join(path, files_path[patch_index]))
            new_data = Image.fromarray(np.uint8(image)).convert('RGB')
            img = transformations(new_data)
            label = int(files_path[patch_index][-5])
            test_dataset.append((img, label, patch_index, imgid, 0))

    train_dataset = np.array(train_dataset)
    labeled_set = np.array(labeled_set)
    unlabeled_set = np.array(unlabeled_set)
    test_dataset = np.array(test_dataset)
    
    print(len(train_dataset), len(labeled_set), len(
        unlabeled_set), len(test_dataset), len(files))

    #
    np.save(os.path.join(save_path, "patch_file.npy"), np.array(files))
    np.save(os.path.join(save_path, "ts_patch_file.npy"), np.array(ts_files))

    np.save(os.path.join(save_path, "train_dataset.npy"), train_dataset)
    np.save(os.path.join(save_path, "init_labeled_set.npy"), labeled_set)
    np.save(os.path.join(save_path, "init_unlabeled_set.npy"), unlabeled_set)
    np.save(os.path.join(save_path, "test_dataset.npy"), test_dataset)


In [4]:
save_img_path = './init_data/'
img_path = './init_images/'

In [5]:
npy_data_init_percent(img_path=img_path,save_path=save_img_path,img_num=18)

100%|██████████| 18/18 [02:55<00:00,  9.77s/it]
/var/folders/th/_3txp6g94497mdxt9g429v5r0000gn/T/ipykernel_2507/3498081373.py:73: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  train_dataset = np.array(train_dataset)
/var/folders/th/_3txp6g94497mdxt9g429v5r0000gn/T/ipykernel_2507/3498081373.py:73: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_dataset = np.array

4336 2713 1623 1092 4336


In [ ]:
"""
好的，我来一行一行地讲解这段代码的作用：

1. `def npy_data_init_percent(`：定义一个名为`npy_data_init_percent`的函数，用于初始化数据集。
2. `img_path, save_path, label_rate=0.5, test_rate=0.2, size=224, img_num=0):`：定义函数的参数，包括图像路径、保存路径、有标签数据比例、测试数据比例、图像大小和图像数量。
3. `if not os.path.exists(save_path):`：如果保存路径不存在，则创建该路径。
4. `os.makedirs(save_path)`：创建保存路径。
5. `train_dataset = []`：定义一个空列表，用于存储训练集的数据。
6. `labeled_set = []`：定义一个空列表，用于存储有标签集的数据。
7. `unlabeled_set = []`：定义一个空列表，用于存储无标签集的数据。
8. `test_dataset = []`：定义一个空列表，用于存储测试集的数据。
9. `files = []`：定义一个空列表，用于存储所有数据的文件名。
10. `ts_files=[]`：定义一个空列表，用于存储测试集的文件名。
11. `transformations = transforms.Compose([`：定义一个数据转换器，用于将图像数据转换为张量，并进行归一化处理。
12. `transforms.Resize(size),`：将图像大小调整为指定大小。
13. `transforms.ToTensor(),`：将图像数据转换为张量。
14. `transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])`：对张量进行归一化处理。
15. `global_index = 0`：定义一个全局索引，用于标记每个数据的唯一编号。
16. `for imgid in tqdm(range(img_num)):`：遍历每个图像文件夹。
17. `files_path = []`：定义一个空列表，用于存储当前图像文件夹中的所有文件名。
18. `path = os.path.join(img_path, 'image_'+str(imgid))`：获取当前图像文件夹的路径。
19. `for file_name in os.listdir(path):`：遍历当前图像文件夹中的所有文件。
20. `lb=int(file_name.split('.')[0][-1])`：从文件名中获取标签。
21. `if lb!=0:`：如果标签不为0，则将文件名添加到`files_path`列表中。
22. `patch_num = len(files_path)`：获取当前图像文件夹中的文件数量。
23. `rand_init = torch.randperm(patch_num)`：生成一个随机排列，用于随机选择有标签数据、无标签数据和测试数据。
24. `labeled_index = rand_init[:int(label_rate*patch_num)]`：根据有标签数据比例，选择有标签数据的索引。
25. `unlabeled_index = rand_init[int(label_rate*patch_num): int((1-test_rate)*patch_num)]`：根据有标签数据比例和测试数据比例，选择无标签数据的索引。
26. `for patch_index in rand_init[:int((1-test_rate)*patch_num)]:`：遍历有标签数据和无标签数据。
27. `files.append((imgid, patch_index, files_path[patch_index]))`：将当前数据的文件名、图像编号和索引添加到`files`列表中。
28. `image = Image.open(os.path.join(path, files_path[patch_index]))`：打开当前数据的图像文件。
29. `new_data = Image.fromarray(np.uint8(image)).convert('RGB')`：将图像数据转换为RGB格式。
30. `img = transformations(new_data)`：将图像数据转换为张量，并进行归一化处理。
31. `label = int(files_path[patch_index][-5])`：从文件名中获取当前数据的标签。
32. `train_dataset.append((img, label, patch_index, imgid, global_index))`：将当前数据的图像数据、标签、索引、图像编号和唯一编号添加到训练集中。
33. `if patch_index in labeled_index:`：如果当前数据的索引在有标签数据的索引中。
34. `labeled_set.append((img, label, patch_index, imgid, global_index))`：将当前数据的图像数据、标签、索引、图像编号和唯一编号添加到有标签集中。
35. `elif patch_index in unlabeled_index:`：如果当前数据的索引在无标签数据的索引中。
36. `unlabeled_set.append((img, label, patch_index, imgid, global_index))`：将当前数据的图像数据、标签、索引、图像编号和唯一编号添加到无标签集中。
37. `global_index += 1`：全局索引加1。
38. `for patch_index in rand_init[int((1-test_rate)*patch_num):]:`：遍历测试数据。
39. `ts_files.append((imgid, patch_index, files_path[patch_index]))`：将当前数据的文件名、图像编号和索引添加到测试集的文件名列表中。
40. `image = Image.open(os.path.join(path, files_path[patch_index]))`：打开当前数据的图像文件。
41. `new_data = Image.fromarray(np.uint8(image)).convert('RGB')`：将图像数据转换为RGB格式。
42. `img = transformations(new_data)`：将图像数据转换为张量，并进行归一化处理。
43. `label = int(files_path[patch_index][-5])`：从文件名中获取当前数据的标签。
44. `test_dataset.append((img, label, patch_index, imgid, 0))`：将当前数据的图像数据、标签、索引、图像编号和0添加到测试集中。
45. `train_dataset = np.array(train_dataset)`：将训练集转换为numpy数组。
46. `labeled_set = np.array(labeled_set)`：将有标签集转换为numpy数组。
47. `unlabeled_set = np.array(unlabeled_set)`：将无标签集转换为numpy数组。
48. `test_dataset = np.array(test_dataset)`：将测试集转换为numpy数组。
49. `print(len(train_dataset), len(labeled_set), len(unlabeled_set), len(test_dataset), len(files))`：打印数据集的大小信息。
50. `np.save(os.path.join(save_path, "patch_file.npy"), np.array(files))`：将所有数据的文件名保存到指定路径中。
51. `np.save(os.path.join(save_path, "ts_patch_file.npy"), np.array(ts_files))`：将测试集的文件名保存到指定路径中。
52. `np.save(os.path.join(save_path, "train_dataset.npy"), train_dataset)`：将训练集保存到指定路径中。
53. `np.save(os.path.join(save_path, "init_labeled_set.npy"), labeled_set)`：将有标签集保存到指定路径中。
54. `np.save(os.path.join(save_path, "init_unlabeled_set.npy"), unlabeled_set)`：将无标签集保存到指定路径中。
55. `np.save(os.path.join(save_path, "test_dataset.npy"), test_dataset)`：将测试集保存到指定路径中。
"""

In [6]:
def npy_data_init_bk(
        img_path,  # /init_data_image
        save_path,  # /init_data
        size=224,
        img_num=0,
):

    if not os.path.exists(save_path):
        os.makedirs(save_path)
    # imglen是wsi图的个数
    bk_files=[]
    bk_data=[]

    transformations = transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

    global_index = 0
    for imgid in tqdm(range(img_num)):
        bk_files_path=[]
        path = os.path.join(img_path, 'image_'+str(imgid+1))

        for file_name in os.listdir(path):
            lb=int(file_name.split('_')[-1][0]) 
            if lb==0:
                bk_files_path.append(file_name)
        #print(imgid,len(bk_files_path))
        # bk
        for patch_index in range(len(bk_files_path)):
            bk_files.append((imgid, patch_index, bk_files_path[patch_index]))
            image = Image.open(os.path.join(path, bk_files_path[patch_index]))
            new_data = Image.fromarray(np.uint8(image)).convert('RGB')
            img = transformations(new_data)
            label = int(bk_files_path[patch_index][-5])
            bk_data.append(
                (img, label, patch_index, imgid, global_index))
    
    bk_data = np.array(bk_data)
    bk_files = np.array(bk_files)

    print(len(bk_data),len(bk_files))
    np.save(os.path.join(save_path, "bk_dataset.npy"), bk_data)
    np.save(os.path.join(save_path, "bk_patch_file.npy"), bk_files)

In [7]:
save_img_path = './init_data/'
img_path = './init_images/'

In [8]:
npy_data_init_bk(
        img_path=img_path,
        save_path=save_img_path,
        img_num=18
    )

100%|██████████| 18/18 [00:35<00:00,  1.94s/it]
/var/folders/th/_3txp6g94497mdxt9g429v5r0000gn/T/ipykernel_2507/289861869.py:39: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  bk_data = np.array(bk_data)
/var/folders/th/_3txp6g94497mdxt9g429v5r0000gn/T/ipykernel_2507/289861869.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bk_data = np.array(bk_data)


3528 3528
